# Fincon Rowing Session on Smart City and IoT
Dieses Beispiel zeigt, wie mit einfachem Code in Python Daten von Wetterstationen in der Region Bern ausgelesen und visualisiert werden können.

API Dokumentation: https://meteotest.github.io/urban-heat-API-docs/de/

Entwickelt im Rahmen des Projektes __[Smart Urban Heat Map Bern](https://smart-urban-heat-map.ch/)__.


## Teil 1 - Laden und Darstellen von aktuellen Daten

### Schritt 1: Importieren der benötigten Python Libraries

In [1]:
import io # for working with streams (like strings)
import requests # for making standard HTTP requests
import pandas as pd # for storing and manipulating data
import geopandas as gpd # for storing and manipulating geographic data
import folium # for creating maps
import branca # for using colorscales in Folium
import json # for data handling

### Schritt 2: Liste aller Messstationen abfragen

In [2]:
# Define the URL
url = "https://smart-urban-heat-map.ch/api/1.0/stations"

# Make the GET request
response = requests.get(url)

# Display the result
print(response.text)

{"type": "FeatureCollection", "features": [{"type": "Feature", "geometry": {"type": "Point", "coordinates": [7.4133654, 46.938255]}, "properties": {"stationId": "D7DD50FEFF6E959E", "name": "Liebefeld Turnierstrasse Laterne", "dateObserved": "2024-05-13T00:21:06Z", "temperature": 12.177081, "relativeHumidity": 86.891205}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [7.42946, 46.94672]}, "properties": {"stationId": "9FD350FEFF6E959E", "name": "Emch+Berger (Schl\u00f6sslistrasse) Pfosten Eingang", "dateObserved": "2024-05-13T00:21:03Z", "temperature": 15.063706, "relativeHumidity": 69.024796}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [7.45326, 46.93122]}, "properties": {"stationId": "3740CBFEFFE70FFE", "name": "Wabern Viktoriastrasse", "dateObserved": "2024-05-13T00:20:57Z", "temperature": 14.20119, "relativeHumidity": 82.63584}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [7.48807, 46.95813]}, "properties": {"stationId"

Umformatierung für bessere Lesbarkeit:

In [3]:
print(json.dumps(response.json(), indent=2))

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          7.4133654,
          46.938255
        ]
      },
      "properties": {
        "stationId": "D7DD50FEFF6E959E",
        "name": "Liebefeld Turnierstrasse Laterne",
        "dateObserved": "2024-05-13T00:21:06Z",
        "temperature": 12.177081,
        "relativeHumidity": 86.891205
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          7.42946,
          46.94672
        ]
      },
      "properties": {
        "stationId": "9FD350FEFF6E959E",
        "name": "Emch+Berger (Schl\u00f6sslistrasse) Pfosten Eingang",
        "dateObserved": "2024-05-13T00:21:03Z",
        "temperature": 15.063706,
        "relativeHumidity": 69.024796
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          7.

Laden der Daten als GeoPandas Dataframe macht das ganze noch einfacher:

In [ ]:
stations = gpd.read_file(response.text)

# Display stations DataFrame
stations

### Schritt 3: Darstellen auf einer Karte

Zuerst zeigen wir eine Karte an, die alle Messstationen abdeckt:


In [ ]:
# Create a base map
map1 = folium.Map(location=[stations.geometry.y.mean(), stations.geometry.x.mean()], zoom_start=10)

# Display map
map1

Jetzt zeichnen wir die Messstationen ein:

In [ ]:
# Create a base map
map2 = folium.Map(location=[stations.geometry.y.mean(), stations.geometry.x.mean()], zoom_start=10)

# Iterate through each station in the GeoPandas DataFrame
for idx, station in stations.iterrows():
    # Create a circle marker for each station and add it to the map
    folium.CircleMarker(
        # Location of the marker in lat and lon
        location=(station.geometry.y, station.geometry.x),
        # Styling the marker
        radius=5,
        color="black",  # Border color
        weight=0.5,  # Border width
        fill=True,
        fill_color="black",
        fill_opacity=1, # 1 means 100% opaque
        # Add a tooltip with station name
        tooltip=f"{station['name']}",
    # Add the marker to the map
    ).add_to(map2)

map2

Und zum Schluss können wir den Messpunkten eine von der aktuellen Temperatur abhängige Farbe geben und auch Tooltips hinzufügen:


In [ ]:
# Create a base map
map3 = folium.Map(location=[stations.geometry.y.mean(), stations.geometry.x.mean()], zoom_start=13)#, tiles="CartoDB positron")

# Most popular colormaps are sequential, meaning that they have a start and end color
# Define a function to reverse a colormap
def reversed_colormap(existing):
    return branca.colormap.LinearColormap(
        colors=list(reversed(existing.colors)),
        vmin=existing.vmin, vmax=existing.vmax
    )

def formatToolTip(stationName, temperature, humidity):
    return f"{stationName}: {temperature:.2f}°C, {humidity:.2f}%"

def formatToolTip2(stationName, temperature, humidity):
    return f"<h4>{stationName}</h4>Temperatur: {temperature:.2f}°C<br/>Luftfeuchtigkeit: {humidity:.2f}%"


# Create a colormap for a temperature range of 10-37°C
# Choose an appropriate colormap (Spectral_11) and reverse it so that it goes from blue (cold) to red (hot)
colormap = reversed_colormap(branca.colormap.linear.Spectral_11)
# Scale the colormap to the temperature range and discretize it into 9 steps for better visibility
colormap = colormap.scale(10, 37).to_step(9)
# Add a caption to the colormap
colormap.caption = "Temperature (°C)"
# Add colormap to the map
colormap.add_to(map3)

# Iterate through each station in the GeoPandas DataFrame
for idx, station in stations.iterrows():
    # Create a circle marker for each station and add it to the map
    folium.CircleMarker(
        # Location of the marker in lon and lat
        location=(station.geometry.y, station.geometry.x),
        # Styling the marker
        radius=5,
        color="black",  # Border color
        weight=0.5,  # Border width
        fill=True,
        # Set the fill color to the colormap value corresponding to the station temperature
        fill_color=colormap(station['temperature']),
        fill_opacity=1, # 1 means 100% opaque
        # Add a tooltip with station name and temperature
        tooltip=formatToolTip2(station['name'], station['temperature'], station['relativeHumidity'])
    # Add the marker to the map
    ).add_to(map3)

# Display map
map3

## Teil 2 - Zeitlichen Messdatenverlauf darstellen

### Schritt 1: Daten zu einer Messstation laden

Z.B. am Bundesplatz:

In [ ]:
# Based on a station name, get the respective stationId from the DataFrame
stationId = stations[stations["name"] == "Bundesplatz"].stationId.values[0]
# Define the URL including the stationId argument
url = f"https://smart-urban-heat-map.ch/api/1.0/timeseries?stationId={stationId}"

# Define the headers to request CSV data as a python dictionary
headers = {
    "Accept": "text/csv"
}

# Make the GET request with the headers
response = requests.get(url, headers=headers)

# Get the timeseries into a data frame
timeseries = pd.read_csv(io.StringIO(response.text), delimiter=";")

timeseries

### Schritt 2: Daten grafisch darstellen

In [ ]:
# Use the DataFrame plot function to plot the temperature time series as a line plot with markers
ax = timeseries.plot(x='dateObserved', y='temperature', figsize=(10,6), marker=None, linestyle='-')
# The ax object is a matplotlib axis object which we can use to further customize the plot

# Add a title
ax.set_title('Bundesplatz')

# Add labels for the x and y axes
ax.set_xlabel('Zeit')
ax.set_ylabel('Temperatur (°C)')

# Shift every second x-label for better readability
for i, label in enumerate(ax.get_xticklabels()):
    if i % 2 == 0:  # Check if it's an odd tick
        label.set_y(-0.04)

### Schritt 3: Daten von mehreren Stationen anzeigen
Zuerst legen wir den Zeitraum fest:

In [ ]:
startTime = "2023-09-28T00:00:00Z" # 28.09.23 00:00 ; Z means UTC time
endTime = "2023-09-30T23:00:00Z" # 31.09.23 23:00

Jetzt lesen wir die Daten:

In [ ]:
import matplotlib.pyplot as plt

# Define function to read the data for a specific statio and a specific time range.
# Returns the data as a Pandas dataframe
def getData(stationName, startTime, endTime):
    #Define the headers to request CSV data as a python dictionary
    headers = {
        "Accept": "text/csv"
    }

    # Read the data
    stationId = stations[stations["name"] == stationName].stationId.values[0]
    # Define the URL including the stationId argument
    url = f"https://smart-urban-heat-map.ch/api/1.0/timeseries?stationId={stationId}&timeFrom={startTime}&timeTo={endTime}"

    # Make the GET request with the headers
    response = requests.get(url, headers=headers)
    return pd.read_csv(io.StringIO(response.text), delimiter=";")

# Read the data for the 3 stations:
timeseries1 = getData("Bundesplatz", startTime, endTime)
timeseries2 = getData("Marktplatz Lyss", startTime, endTime)
timeseries3 = getData("Sigriswil", startTime, endTime)

# Now plot the data
plt.figure(figsize=(10, 6))


# Use the DataFrame plot function to plot the temperature time series as a line plot with markers
timeseries1.plot(x='dateObserved', y='temperature', label="Bundesplatz", marker=None, color='blue', linestyle='-', ax=plt.gca())
timeseries2.plot(x='dateObserved', y='temperature', label="Marktplatz Lyss", marker=None, color='green', linestyle='-', ax=plt.gca())
timeseries3.plot(x='dateObserved', y='temperature', label="Sigriswil", marker=None, color='red', linestyle='-', ax=plt.gca())

plt.title('Temperaturverlauf')
plt.xlabel('Zeit')
plt.ylabel('Temperatur (°C)')



